In [84]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

In [85]:
    
def parse_team_data(three_letter_ID):
    driver = webdriver.Chrome()
    chrome_options = Options()
    chrome_options.headless = True
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(3)
    driver.get(f"https://www.basketball-reference.com/teams/{three_letter_ID}/2022.html")
    roster_table = driver.find_element(By.ID, "roster")
    raw_html = roster_table.get_attribute('innerHTML')
    driver.close()

    soup = BeautifulSoup(raw_html, 'html.parser')

    player_data = soup.findAll('tr')
    player_dict = []
    del player_data[0]


    keys = ['Player_Name','Position','Height','Weight','DOB','COB','Experience','Alma_Matter']
    for player in player_data:
        found_rows= player.find_all('td')
        stripped_text = [ele.text for ele in found_rows]

        unfinished_dict = dict(zip(keys,stripped_text))
        unfinished_dict['Number'] = player.findAll('th',attrs={'scope':'row'})[0].getText()
        player_dict.append(unfinished_dict)


    return (player_dict,player_data)

In [86]:
def fetch_contract_data(player_data,three_letter_ID):
    chrome_options = Options()
    chrome_options.headless = True
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(3)
    raw_contract_data = []
    for player in player_data:
        get_html = player.a['href']
        driver = webdriver.Chrome(options=chrome_options)
        driver.get('https://www.basketball-reference.com/'+get_html)
        element_to_find = f'contracts_{three_letter_ID.lower()}'
        try:
            salary_table = driver.find_element(By.ID,element_to_find)          
            raw_html = salary_table.get_attribute('innerHTML')
            soup = BeautifulSoup(raw_html, 'html.parser')
            y = soup.findAll('tr',attrs={'data-row':0})
            contracts = soup.findAll('td',attrs={'class_':""})
            del contracts[0]
            raw_contract_data.append(contracts)
        except:
            raw_contract_data.append(['NaN'])
        driver.close()        
    return raw_contract_data

In [87]:
def merge_contract_data_with_player_data(raw_contract_data,player_dict):   
    player_salaries = []
    for player in raw_contract_data:
        starting_year = '2021-2022'
        player_salary = []
        counter = 0
        for row in player:
            if row == "NaN":
                player_salary.append({starting_year:'NaN'})
                counter+=1
            else:              
                current_contract = {}
                year_parsed = BeautifulSoup(str(row), 'html.parser')                          
                year_fix = list(starting_year)
                year_fix[3] = int(year_fix[3]) + counter
                year_fix[-1] = int(year_fix[-1]) + counter
                year_fix[3] = str(year_fix[3])
                year_fix[-1] = str(year_fix[-1])
                year_fix= "".join(year_fix)
                current_contract[year_fix] =  year_parsed.getText()
                player_salary.append(current_contract)
                counter+=1
        player_salaries.append(player_salary)            
    counter = 0
    copy_of_dict = player_dict
    for player in copy_of_dict:
        player_salaries_by_year = player_salaries[counter]
        for x in player_salaries_by_year[0].values():
            check_x = x
        if check_x=='NaN':
            player['Salary '+ contract_year] = 'NaN'
        else:
            for contract in player_salaries_by_year:
                contract_year = ''
                for x in contract.keys():
                    contract_year=x
                contract_value = 0
                for x in contract.values():
                    contract_value = x
                player['Salary '+ contract_year] = contract_value
            counter+=1
    return copy_of_dict


In [88]:

merge_contract_data_with_player_data(team_data[0][1],team_data[0][0])

[{'Player_Name': 'Duncan Robinson',
  'Position': 'SG',
  'Height': '6-7',
  'Weight': '215',
  'DOB': 'April 22, 1994',
  'COB': 'us',
  'Experience': '3',
  'Alma_Matter': 'Williams, Michigan',
  'Number': '55',
  'Salary 2021-2022': '$15,560,000',
  'Salary 2022-2023': '$16,902,000',
  'Salary 2023-2024': '$18,154,000',
  'Salary 2024-2025': '$19,406,000',
  'Salary 2025-2026': '$19,888,000'},
 {'Player_Name': 'P.J. Tucker',
  'Position': 'PF',
  'Height': '6-5',
  'Weight': '245',
  'DOB': 'May 5, 1985',
  'COB': 'us',
  'Experience': '10',
  'Alma_Matter': 'Texas',
  'Number': '17',
  'Salary 2021-2022': '$7,000,000',
  'Salary 2022-2023': '$7,350,000'},
 {'Player_Name': 'Gabe Vincent',
  'Position': 'PG',
  'Height': '6-3',
  'Weight': '200',
  'DOB': 'June 14, 1996',
  'COB': 'us',
  'Experience': '2',
  'Alma_Matter': 'UC Santa Barbara',
  'Number': '2',
  'Salary 2021-2022': '$1,669,178',
  'Salary 2022-2023': '$1,815,677'},
 {'Player_Name': 'Dewayne Dedmon',
  'Position': 'C'

In [89]:
three_letter_codes = ['MIA' ,'MIL','PHI','BOS','CHI','CLE','TOR',
                      'BRK' ,'CHO' ,'ATL' ,'WAS' ,'NYK' ,'IND' ,'DET' ,'ORL' ,'PHO' 
                      ,'MEM' ,'GSW' ,'UTA' ,'DAL' ,'DEN' ,'MIN' ,'LAC' ,'LAL' ,'NOP' 
                      ,'SAS' ,'POR' ,'SAC','OKC','HOU'] 


In [ ]:
team_data = []
for team in three_letter_codes:
    player_dict , player_data = parse_team_data(team)
    raw_contract_data = fetch_contract_data(player_data,team)
    team_data.append((player_dict,raw_contract_data))
    print(f'Finished {team}')

    

Finished MIA
Finished MIL
Finished PHI
Finished BOS
Finished CHI
Finished CLE
Finished TOR
Finished BRK
Finished CHO
Finished ATL
Finished WAS
Finished NYK
Finished IND
Finished DET
Finished ORL
Finished PHO
Finished MEM


In [79]:
fixed_team_data = []

counter = 0

for team in team_data:
    x,y = team_data[counter][0],team_data[counter][1]
    fixed_team_data.append(merge_contract_data_with_player_data(y,x))
    counter+=1


In [82]:
fixed_team_data[0]

[{'Player_Name': 'Duncan Robinson',
  'Position': 'SG',
  'Height': '6-7',
  'Weight': '215',
  'DOB': 'April 22, 1994',
  'COB': 'us',
  'Experience': '3',
  'Alma_Matter': 'Williams, Michigan',
  'Number': '55',
  'Salary 2021-2022': '$15,560,000',
  'Salary 2022-2023': '$16,902,000',
  'Salary 2023-2024': '$18,154,000',
  'Salary 2024-2025': '$19,406,000',
  'Salary 2025-2026': '$19,888,000'},
 {'Player_Name': 'P.J. Tucker',
  'Position': 'PF',
  'Height': '6-5',
  'Weight': '245',
  'DOB': 'May 5, 1985',
  'COB': 'us',
  'Experience': '10',
  'Alma_Matter': 'Texas',
  'Number': '17',
  'Salary 2021-2022': '$7,000,000',
  'Salary 2022-2023': '$7,350,000'},
 {'Player_Name': 'Gabe Vincent',
  'Position': 'PG',
  'Height': '6-3',
  'Weight': '200',
  'DOB': 'June 14, 1996',
  'COB': 'us',
  'Experience': '2',
  'Alma_Matter': 'UC Santa Barbara',
  'Number': '2',
  'Salary 2021-2022': '$1,669,178',
  'Salary 2022-2023': '$1,815,677'},
 {'Player_Name': 'Dewayne Dedmon',
  'Position': 'C'

In [38]:
team_data[0][1][1]

[<td><span class="">$7,000,000</span></td>,
 <td><span class="salary-pl">$7,350,000</span></td>]

In [55]:
team_data

[([{'Player_Name': 'Duncan Robinson',
    'Position': 'SG',
    'Height': '6-7',
    'Weight': '215',
    'DOB': 'April 22, 1994',
    'COB': 'us',
    'Experience': '3',
    'Alma_Matter': 'Williams, Michigan',
    'Number': '55',
    'Salary 2021-2022': '$15,560,000',
    'Salary 2022-2023': '$16,902,000',
    'Salary 2023-2024': '$18,154,000',
    'Salary 2024-2025': '$19,406,000',
    'Salary 2025-2026': '$19,888,000'},
   {'Player_Name': 'P.J. Tucker',
    'Position': 'PF',
    'Height': '6-5',
    'Weight': '245',
    'DOB': 'May 5, 1985',
    'COB': 'us',
    'Experience': '10',
    'Alma_Matter': 'Texas',
    'Number': '17',
    'Salary 2021-2022': '$7,000,000',
    'Salary 2022-2023': '$7,350,000',
    'Salary 2023-2024': '$18,154,000',
    'Salary 2024-2025': '$19,406,000',
    'Salary 2025-2026': '$19,888,000',
    'Salary 2026-2027': '$7,000,000',
    'Salary 2027-2028': '$7,350,000'},
   {'Player_Name': 'Gabe Vincent',
    'Position': 'PG',
    'Height': '6-3',
    'Weight